Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

Let's try to make a Gaussian node with several different sizes and parameterizations:

In [2]:
ndims = (5, 5)  # node dimensions
vardims = 3  # dimension of variable

3

In [3]:
μ = Array{Vector{Float64}}(ndims)
Σ = Array{Matrix{Float64}}(ndims)

for idx in eachindex(μ)
    μ[idx] = rand(vardims)
end

for idx in eachindex(Σ)
    C = rand(vardims, vardims)
    Σ[idx] = C * C'
end

In [4]:
μ[1]

3-element Array{Float64,1}:
 0.342152
 0.807456
 0.284293

In [5]:
Σ[1]

3x3 Array{Float64,2}:
 0.593245  0.830263  1.06591
 0.830263  1.19847   1.51821
 1.06591   1.51821   2.26962

In [6]:
det(Σ[2])

0.10590065436887316

In [7]:
pars = (μ, Σ)
aa = map(MultivariateNormal, pars...);

In [8]:
rr = RandomNode(aa, Dict{Factor, Symbol}());

In [9]:
typeof(aa)

Array{Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}},2}

In [10]:
typeof(aa) <: AbstractArray{typeof(aa[1]), 2}

true

In [11]:
ss = RandomNode(MultivariateNormal, pars...);

In [12]:
E(ss)

5x5 Array{Array{Float64,1},2}:
 [0.34215233603981177,0.8074560786750042,0.2842928356430334]  …  [0.9538131675898092,0.4973503713844143,0.2937770320890263] 
 [0.9106150594184297,0.36695611162038655,0.8514679376028791]     [0.6826949327432597,0.4805670946347982,0.6838657733633682] 
 [0.10146673940594697,0.9623853650154,0.3117973636792539]        [0.7997495283730063,0.7936112065852754,0.7180790565621349] 
 [0.5890465429139773,0.42299866882358894,0.6272023931839328]     [0.02659813167230718,0.3509057828334454,0.2989353830726533]
 [0.2976099689702967,0.7854265405616578,0.13366587258961715]     [0.1659879999032896,0.8807105459384512,0.30647194822104207]

In [13]:
typeof(ss).parameters[1]

Distributions.MvNormal{PDMats.PDMat,Array{Float64,1}}

# Make a factor and check its value

In [14]:
μ = rand(10)
τ = rand(10)
x = RandomNode(Normal, rand(10), rand(10))
f = LogNormalFactor(x, μ, τ)

VB.LogNormalFactor(VB.RandomNode{Distributions.Normal,1}([Distributions.Normal(μ=0.3791717231327607, σ=0.4950498125895646),Distributions.Normal(μ=0.47401377458559724, σ=0.28005685414543335),Distributions.Normal(μ=0.6628776249320709, σ=0.9356599248007591),Distributions.Normal(μ=0.09829603246343921, σ=0.0698481276239753),Distributions.Normal(μ=0.7267121098908691, σ=0.5190232244266098),Distributions.Normal(μ=0.6308301093795148, σ=0.08353937201148876),Distributions.Normal(μ=0.07954039604285268, σ=0.9551142460374911),Distributions.Normal(μ=0.8269505435403663, σ=0.006214525667451731),Distributions.Normal(μ=0.10534174301229737, σ=0.29348889850105664),Distributions.Normal(μ=0.0868426258074475, σ=0.4250961222635201)],Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,1}([0.8883642147362805,0.518217061900407,0.7713318826195028,0.8784813671955971,0.7181655179908251,0.2698738696488878,0.7730017766158093,0.296558707835596,0.5932182655347729,0.6040277445849671],Dict{VB.Factor,Symbol}()),VB.ConstantNo

In [15]:
value(f)

-5.649845239392106

In [16]:
register(f)
x.factormap

Dict{VB.Factor,Symbol} with 1 entry:
  VB.LogNormalFactor(VB.R… => :x

In [17]:
g = EntropyFactor(x)
value(g)

-0.8607989098311477

In [18]:
naturals(x)

10-element Array{Tuple{Float64,Float64},1}:
 (1.5471703761512972,-2.0401974643156358) 
 (6.043639486141973,-6.374961878930181)   
 (0.757176806934196,-0.571128650640296)   
 (20.147745261942138,-102.48503808857191) 
 (2.697669951582343,-1.8560788480512966)  
 (90.39200121450814,-71.64528124966786)   
 (0.08719209311601347,-0.5480994403713958)
 (21412.312094200803,-12946.549380407772) 
 (1.2229737726592287,-5.804791802792087)  
 (0.48057221719750265,-2.7669143622111054)

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [19]:
dims = (5, 6)

# note: it won't matter much how we initialize here
μ = RandomNode(Normal, zeros(dims), ones(dims))
τ = RandomNode(Gamma, ones(dims), ones(dims))

y = rand(dims)

5x6 Array{Float64,2}:
 0.019526  0.64192   0.920968   0.221961  0.497111  0.376147
 0.645838  0.886014  0.635094   0.86881   0.173711  0.234695
 0.571483  0.605633  0.240604   0.252273  0.424832  0.433595
 0.246642  0.580497  0.81434    0.503185  0.399902  0.499845
 0.8564    0.315123  0.0795404  0.555742  0.340429  0.753323

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [20]:
obs = LogNormalFactor(y, μ, τ)
μ_prior = LogNormalFactor(μ, zeros(dims), 2 * ones(dims))
τ_prior = LogGammaFactor(τ, 1.1 * ones(dims), ones(dims))

VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,Dict{VB.Factor,Symbol}()))

In [21]:
value(obs), value(μ_prior), value(τ_prior)

(-38.35125483195037,-67.96536370453937,7.496173237795199)

In [22]:
m = VBModel(Dict(:μ => μ, :τ => τ), [obs, μ_prior, τ_prior])

VB.VBModel(Dict{Symbol,VB.Node}(:τ=>VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}(VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(#= circular reference =#),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,

In [23]:
m.nodes[:μ].factormap

Dict{VB.Factor,Symbol} with 2 entries:
  VB.LogNormalFactor(VB.C… => :μ
  VB.LogNormalFactor(VB.R… => :x

In [24]:
naturals(obs, Val{:x}, μ)

(
5x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [25]:
naturals(obs, Val{:μ}, μ)

(
5x6 Array{Float64,2}:
 0.019526  0.64192   0.920968   0.221961  0.497111  0.376147
 0.645838  0.886014  0.635094   0.86881   0.173711  0.234695
 0.571483  0.605633  0.240604   0.252273  0.424832  0.433595
 0.246642  0.580497  0.81434    0.503185  0.399902  0.499845
 0.8564    0.315123  0.0795404  0.555742  0.340429  0.753323,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [26]:
naturals(obs, Val{:τ}, τ)

(0.5,
5x6 Array{Float64,2}:
 0.500191  0.706031  0.924091  0.524633  0.62356   0.570743
 0.708553  0.89251   0.701672  0.877416  0.515088  0.527541
 0.663296  0.683396  0.528945  0.531821  0.590241  0.594002
 0.530416  0.668488  0.831575  0.626598  0.579961  0.624922
 0.86671   0.549651  0.503163  0.654425  0.557946  0.783748)

In [27]:
method_exists(naturals, (LogNormalFactor, Type{Val{:x}}, RandomNode{Normal}))

true

In [28]:
check_conjugate(m.nodes[:τ])

true

In [29]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.0, σ=1.0)  …  Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)

In [30]:
update!(μ, Val{:conjugate})

In [31]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=-0.33974642587876164, σ=0.7071067811865476)  …  Distributions.Normal(μ=-0.08757735964789944, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.1031230083822061, σ=0.7071067811865476)       Distributions.Normal(μ=-0.18759892024976002, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.05054604661844582, σ=0.7071067811865476)      Distributions.Normal(μ=-0.046955363717725414, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.17915149839257358, σ=0.7071067811865476)     Distributions.Normal(μ=-0.00010971345274074199, σ=0.7071067811865476)
 Distributions.Normal(μ=0.2520128176967853, σ=0.7071067811865476)       Distributions.Normal(μ=0.17912620274862429, σ=0.7071067811865476)    